<a href="https://colab.research.google.com/github/napatswift/Data-structures-in-C/blob/main/colab-test/budget_extracting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pdfminer.six
!pip install pdfplumber

In [155]:
from pdfminer.high_level import extract_text, extract_pages
from pdfminer.layout import LAParams, LTTextContainer
import re
import pdfplumber
import pandas as pd
import numpy as np

In [5]:
file_path = '/content/16.pdf'
pdf = pdfplumber.open(file_path)

In [24]:
def get_text_from_page(num):
  w = pdf.pages[num].width
  h = pdf.pages[num].height
  text = pdf.pages[num].within_bbox((0,0,w,h), relative=False).extract_text()
  return text

In [15]:
repl = {'รายละเอียดงบประมาณจ าแนกตามงบรายจ่าย': 'รายละเอียดงบประมาณจำแนกตามงบรายจ่าย',
        'บ ารุง':'บำรุง', 'ส�านัก': 'สำนัก' ,'อื\n่น':"อื่น", 'ด าเนิน':'ดำเนิน', "ที\n่":'ที่', 'เริ\n่มต้น': 'เริ่มต้น',
        'ตัวชี\n้วัด':'ตัวชี้วัด', 'เป้ าหมาย':'เป้าหมาย', 'พื\n้นฐาน':'พื้นฐาน', 'จ าแนก':'จำแนก',
        'บิ\n๊กดาต้า':'บิ๊กกาต้า', 'ก าหนด':'กำหนด', 'ผลสัมฤทธิ\n์': 'ผลสัมฤทธิ์', 'อ าเภอ' : 'อำเภอ' ,
        'ต าบล': 'ตำบล', 'ก ากับดูแล':'กำกับดูแล','ประจ า':'ประจำ', 'ที่ท าการ':'ที่ทำการ',
        'ส านักงาน':'สำนักงาน', 'จัดท า':'จัดทำ', 'กรัมปรัะชาสำัมพัันัธ์์':'กรมประชาสัมพันธ์',
        'สำ�นักง�น':'สำนักงาน', '\t ':'\t','ผู\n้ปฏิบัติงาน':'ผู้ปฏิบัติงาน', 'เพิ\n่ม':'เพิ่ม' , '':'' ,
        'ล้า้าลำน':'ล่วงล้ำลำน้ำ','ล า':'ลำ', 'ส า':'สำ','ส�านักงา':'สำนักงาน',
        'สำำานััก':'สำนัก', 'ก�ร':'การ','มห�ชน':'มหาชน', 'เครื\n่อง':'เครื่อง', 'กระที่รวง':'กระทรวง' , '':'' ,
        'เบี\n้ย':'เบี้ย', 'ค่าเบี\n้ยเลี\n้ยง':'ค่าเบี้ยเลี้ยง', 'ทั\n้งสิ\n้น':'ทั้งสิ้น', 'ตั\n้งงบ':'ตั้งงบ', 'ที่่�':'ที่'}

def rep_t(Str):
  if Str:
    for a, b in repl.items():
      Str = Str.replace(a, b)
    return Str
  else:
    return ''

def tn_to_arb(line):
  num = ''
  for char in line:
    if char == '(':
      break
    if ord(char) >= ord('๐') and ord(char) <= ord('๙'):
      num += str(ord(char) - ord('๐'))
  return int(num)
  
# fixed_pages = [rep_t(x) for x in pages]

In [72]:
def get_doc_info():
  info = {'FY': 0, 'ฉบับ': 0, 'เล่ม': 0}

  for i in range(4):
    current_page = rep_t(get_text_from_page(i))
    if current_page and 'เอกสารงบประมาณ' in current_page and 'ฉบับ' in current_page:
      current_page = current_page.split('\n')
      issue = ''

      for line in current_page[:-1]:
        if 'ฉบับ' in line:
          info['ฉบับ'] = tn_to_arb(line)
        if 'พ.ศ.' in line:
          info['FY'] = tn_to_arb(line) - 543
        if 'เล่่ม' in line or 'เล่ม' in line:
          info['เล่ม'] = tn_to_arb(line)
        prev_line = line
      break

  id_pages = []
  for i in range(30):
    page = rep_t(get_text_from_page(i))
    if page: 
      text = ''.join(get_text_from_page(i).split())
    if re.findall('\d+(ถึง|ถึึง)\d+', text):
      id_pages.append(i)

  page_rage = []
  ministry = ''
  department = ''
  ministries = []

  for i in id_pages:
    page = rep_t(get_text_from_page(i))
    lines = [' '.join(x.split()) for x in page.split('\n')]

    for l in lines:
      if l.startswith('กระทรวง') or l.startswith('องค์การ'):
        ministry = l.split(' ')[0]
        ministries.append(ministry)

      if re.match('\(\d+', l):
        if department:
          page_rage.append(department)
        start_page = [x for x in l.split(" ") if x.isdigit()]
        if start_page:
          start_page = l.find(start_page[0]) - 1
        else:
          start_page = -1
        department = ministry + ',' + l[l.find(') ')+2:start_page]
      if ('7. รายละ' in l or '8. รายงานสถานะ' in l):
        department += ',' + l.split(' ')[-1]

  page_rage.append(department)
  info['PAGE_RANGE'] = page_rage
  info['INDEX_PAGE'] = id_pages
  info['MINISTRY'] = ministries
  
  return info

info = get_doc_info()

In [73]:
info

{'FY': 2022,
 'INDEX_PAGE': [4, 5, 6, 7, 8, 9, 10, 11],
 'MINISTRY': ['กระทรวงคมนาคม', 'กระทรวงดิจิทัลเพื่อเศรษฐกิจและสังคม'],
 'PAGE_RANGE': ['กระทรวงคมนาคม,สำนักงานปลัดกระทรวงคมนาคม,14,30',
  'กระทรวงคมนาคม,กรมการขนส่งทางบก,35,60',
  'กระทรวงคมนาคม,กรมการขนส่งทางราง,66,74',
  'กระทรวงคมนาคม,กรมเจ้าท่า,81,112',
  'กระทรวงคมนาคม,กรมทางหลวง,118,328',
  'กระทรวงคมนาคม,กรมทางหลวงชนบท,335,450',
  'กระทรวงคมนาคม,กรมท่าอากาศยาน,455,481',
  'กระทรวงคมนาคม,สำนักงานนโยบายและแผนการขนส่งและจราจร,487,500',
  'กระทรวงดิจิทัลเพื่อเศรษฐกิจและสังคม,สำนักงานปลัดกระทรวงดิจิทัลเพื่อเศรษฐกิจและสังคม,517,539',
  'กระทรวงดิจิทัลเพื่อเศรษฐกิจและสังคม,กรมอุตุนิยมวิทยา,546,581',
  'กระทรวงดิจิทัลเพื่อเศรษฐกิจและสังคม,สำนักงานคณะกรรมการดิจิทัลเพื่อเศรษฐกิจและสังคมแห่งชาต ิ,590,610',
  'กระทรวงดิจิทัลเพื่อเศรษฐกิจและสังคม,สำนักงานสถิติแห่งชาติ,616,631',
  'กระทรวงดิจิทัลเพื่อเศรษฐกิจและสังคม,สำนักงานพัฒนาธุรกรรมทางอิเล็กทรอนิกส์,642,656',
  'กระทรวงดิจิทัลเพื่อเศรษฐกิจและสังคม,สำนักงานส่งเสริมเศรษฐกิจดิจิทัล,668

In [94]:
def get_rage_in_ranges(sid, start, end):
  all_pages = [get_text_from_page(p) for p in range(len(pdf.pages))]
  start += sid
  end += sid
  dict_of_range = {i : _ for i, _ in enumerate(all_pages) if _ and ('หน่วย : ลา้นบาท' not in _ and
                                              '7. รายละ' not in _ and
                                              '7.2  แผนงาน' not in _ and
                                              'วัตถุประสงค์' not in _ and
                                              'หน่วยนับ' not in _ and i >= start and i < end
                                              )
  }
  return dict_of_range

In [ ]:
list(get_content_in_range(ids_id[-1],17,31).keys())

[28, 29, 30, 31, 32, 34, 36, 38, 40]

In [144]:
def get_range():
  prev = ''
  r = {}
  smd = {}
  for x in info['PAGE_RANGE']:
    m, sm, s, e = x.split(',')
    lts = list(get_rage_in_ranges(info['INDEX_PAGE'][-1], int(s), int(e)).keys())
    if prev and prev == m:
      smd[sm] = lts
      r[m] = smd
    else:
      prev = m
      smd = {}
      smd[sm] = lts
  return r

In [ ]:
dic = get_range()
for mi in dic:
  print(mi)
  for budmi in dic[mi]:
    print(budmi)
    for i in dic[mi][budmi]:
      print(get_text_from_page(i))

In [88]:
info

{'FY': 2022,
 'INDEX_PAGE': [4, 5, 6, 7, 8, 9, 10, 11],
 'MINISTRY': ['กระทรวงคมนาคม', 'กระทรวงดิจิทัลเพื่อเศรษฐกิจและสังคม'],
 'PAGE_RANGE': ['กระทรวงคมนาคม,สำนักงานปลัดกระทรวงคมนาคม,14,30',
  'กระทรวงคมนาคม,กรมการขนส่งทางบก,35,60',
  'กระทรวงคมนาคม,กรมการขนส่งทางราง,66,74',
  'กระทรวงคมนาคม,กรมเจ้าท่า,81,112',
  'กระทรวงคมนาคม,กรมทางหลวง,118,328',
  'กระทรวงคมนาคม,กรมทางหลวงชนบท,335,450',
  'กระทรวงคมนาคม,กรมท่าอากาศยาน,455,481',
  'กระทรวงคมนาคม,สำนักงานนโยบายและแผนการขนส่งและจราจร,487,500',
  'กระทรวงดิจิทัลเพื่อเศรษฐกิจและสังคม,สำนักงานปลัดกระทรวงดิจิทัลเพื่อเศรษฐกิจและสังคม,517,539',
  'กระทรวงดิจิทัลเพื่อเศรษฐกิจและสังคม,กรมอุตุนิยมวิทยา,546,581',
  'กระทรวงดิจิทัลเพื่อเศรษฐกิจและสังคม,สำนักงานคณะกรรมการดิจิทัลเพื่อเศรษฐกิจและสังคมแห่งชาต ิ,590,610',
  'กระทรวงดิจิทัลเพื่อเศรษฐกิจและสังคม,สำนักงานสถิติแห่งชาติ,616,631',
  'กระทรวงดิจิทัลเพื่อเศรษฐกิจและสังคม,สำนักงานพัฒนาธุรกรรมทางอิเล็กทรอนิกส์,642,656',
  'กระทรวงดิจิทัลเพื่อเศรษฐกิจและสังคม,สำนักงานส่งเสริมเศรษฐกิจดิจิทัล,668

In [ ]:
print()